In [ ]:
from __future__ import annotations
import numpy as np
from sympy import Symbol,solve,Eq
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Iterable, Callable
from itertools import combinations
from math import exp
from scipy.optimize import linprog
from numpy.typing import NDArray
from numpy.linalg import LinAlgError
from typing import Optional

In [ ]:
#Vaccine Problem


Q2: Optimizing with spot rates

In [ ]:
def cash_flow_present_value(cash_flow, effective_rate):
    """Enter effective rate in percentage"""
    if isinstance(effective_rate, list) or isinstance(effective_rate, tuple):
        spr = [1] + [1 + r/100 for r in effective_rate]
        p = 0.
        for i, ai in enumerate(cash_flow):
            p += ai / (spr[i]) ** i
        return p
    r = 1 + effective_rate / 100
    res = 0
    for i, ai in enumerate(cash_flow):
        res += ai / r ** i
    return res

class Bond:
    """Implementation of a bond. A face value $30000, 9% bond that matures in 30 periods (years) that pays its coupons twice a period (year) should be entered as Bond(9, 30, 2, 300)"""
    def __init__(self, coupon_percentage, maturity_years, coupon_freq, face_value):
        self.percentage = coupon_percentage / 100
        self.maturity_years = maturity_years
        self.coupon_freq = coupon_freq
        self.face_value = face_value
        flow = [0] + [coupon_percentage / coupon_freq] * maturity_years * coupon_freq
        flow[-1] += face_value
        self.flow = tuple(flow)
    
    @property
    def coupon_payment(self):
        return self.percentage * self.face_value / self.coupon_freq

def QM_duration(cash_flow_or_bond, rates, compound_freq):
    """Enter a list of spot rates (in percentage) to calculate the quasi-modified duration.
    Keep in mind if the price at rates = sk + y is P(y), then P'(0) = -Dq * P(0)

    This trims the first term to keep consistent with the bond calculations. i.e. following the book. calculating the
    quasimodified duration for (x0, x1, ..., xn) where x0 is the flow on year 0
    
    You can actually feed a bond into this function and we will handle it for you"""
    if isinstance(cash_flow_or_bond, Bond):
        cash_flow = cash_flow_or_bond.flow
    else:
        cash_flow = cash_flow_or_bond
    
    assert len(cash_flow) <= len(rates) + 1, f"Wrong number of periods: we have {len(rates)} years of spot rates but {len(cash_flow)} years bond"

    # First calculate the pv of cash flow
    spr = [1] + [1 + r/100 for r in rates]
    res = 0
    for i, ai in enumerate(cash_flow):
        res += ai / (spr[i]) ** i
    PV = res

    x = cash_flow[1:]
    s = [1 + r/100 for r in rates]
    m = compound_freq
    n = len(cash_flow) - 1
    res = 0.
    for _k in range(n):
        k = _k + 1
        rs = (k/m) * x[_k] * (s[_k]) ** (-k-1)
        res += rs
    return res/PV

def immunize_parallel(bond1: Bond, bond2: Bond, obligation_cash_flow, rates, verbose: bool = False):
    # Calculate the PVob, Dob of obligation
    # Calculate the PV1, D1 of bond1
    # Calculate the PV2, D2 of bond2

    obligation = obligation_cash_flow

    # First do error checking
    num_years_total = max(bond1.maturity_years, bond2.maturity_years, len(obligation) - 1)
    enough_rates = len(rates) >= num_years_total
    if not enough_rates:
        raise ValueError(f"Expected at least {num_years_total} years of rates, found {len(rates)}")

    PVob = cash_flow_present_value(obligation, rates)
    PV1 = cash_flow_present_value(bond1.flow, rates)
    PV2 = cash_flow_present_value(bond2.flow, rates)
    
    Dob = QM_duration(obligation, rates, 1)
    D1 = QM_duration(bond1, rates, 1)
    D2 = QM_duration(bond2, rates, 1)

    if verbose:
        print(f"Duration of bond 1: {D1}")
        print(f"Duration of bond 2: {D2}")
        print(f"Duration of obligation: {Dob}")
        print(f"Present value of bond 1: {PV1}")
        print(f"Present value of bond 2: {PV2}")
        print(f"Present value of obligation: {PVob}")

    # Solve for
    # PV1   PV2     x1 = PVob
    # PV1D1 PV2D2   x2 = PVob Dob

    # Hence
    # x1 =  |PV1   PV2  |-1  |PVob    |
    # x2 =  |PV1D1 PV2D2|    |PVob Dob|
    det = PV1 * PV2 * D2 - PV2 * PV1 * D1
    if abs(det) < 1e-5:
        raise ValueError("This portfolio cannot be immunized - the equations have no solutions")

    x1 = 1/det * (PV2 * PVob * D2 - PVob * Dob * PV2)
    x2 = 1/det * (PV1 * PVob * Dob - PVob * D1 * PV1)

    if verbose:
        print(f"Buy {x1} units of bond 1 and {x2} units of bond 2")

    return x1, x2


## The inputs: in percentage
spot_rates = [2.25, 2.50, 2.75, 3.00, 3.25]
bond1 = Bond(6, 5, 1, 100)
bond2 = Bond(10, 4, 1, 100)
obligation = (0, 50000, 51000 + 51000, 52000 + 52000, 53000 + 53000, 54000)
immunize_parallel(bond1, bond2, obligation, spot_rates)

(-2033.5124975117003, 4842.659952631226)

Q3: Treasury inflation protected security

In [ ]:
## Just build a zero coupon bond
# Bond 1 has cash flow (0, 3, ..., 3, 103)
# Bond 2 has cash flow (0, 6, ..., 6, 106)
# 2*bond 1 - bond 2 = 100 so we dont need to multiply any of the CPI stuff
# Hence price is 2 * 79.66 - 100 = 59.32

Question 4 mortgage


In [ ]:
def new_paymt(P, n1, m, r1, r2, n2): 
    A1 = r1/m*P*(1-1/(1+r1/m)**(n1*m))**(-1)
    print('Orginal Payment:', A1)
    P1 = A1/(r1/m)*(1-1/(1+r1/m)**((n1-n2)*m))
    A2 = r2/m*P1*(1-1/(1+r2/m)**((n1-n2)*m))**(-1)
    return A2

new_paymt(P=1000000, n1=30, m=12, r1=0.02, r2=0.05, n2=5)

Orginal Payment: 3696.194726888144


5097.880172055114

Quetion 5 Mortegage extention

In [ ]:
import math
#P = mortegage value n1 = original terms, n2= rest terms, r1 = ir1, r2 =ir2
def ext_term(P, n1, m, r1, r2, n2): 
    A1 = r1/m*P*(1-1/(1+r1/m)**(n1*m))**(-1)
    P1 = A1/(r1/m)*(1-1/(1+r1/m)**((n1-n2)*m))
    a = P1 * r2/m / A1
    b = 1 - a
    t = -math.log(b, (1+r2/m))
    return t - (n1-n2)*m

ext_term(P=1000000, n1=30, m=12, r1=0.02, r2=0.05, n2=5)

680.5225038849858

Qeustion 8 CAPM


In [ ]:
def allocate_a(n, rb, rm, rf):   # $n dollars, a in market portfolio, n-a in risk-free asset, return a
    # n*rb = a*rm + (n-a)*rf
    return n*(rb-rf) / (rm-rf)

allocate_a(n=1000, rb=0.18, rm=0.12, rf=0.02)

1600.0000000000002

Q9-13: Markoweats my ass

In [ ]:
class OptimizeStockPortfolio:
    def __init__(self, expected_returns: list[float], covariances: NDArray[np.float64]):
        """Solves the Markovitz problem. Refer to the example below"""
        r, w = covariances.shape
        self.n = len(expected_returns)

        if r != w or r != self.n:
            raise ValueError(f"Expected number of stocks equal number of correlations... got number of stocks: {self.n}, correlation array: {r}x{w}")

        if not np.all(covariances - covariances.T < 1e-6): #type: ignore
            raise ValueError(f"Expected the covariance to be a symmetric matrix")

        # if np.all(np.abs(covariances.diagonal() - 1) < 1e-6): #type: ignore
        #     print("All diagonal entries are 1, did you enter the correlation matrix?")

        self.expecteds = np.array(expected_returns)
        self.covariances = np.array(covariances)

    def returns(self, weights: NDArray[np.float64]) -> tuple[float,float]: #type: ignore
        r = weights.dot(self.expecteds)
        sd = weights.T @ self.covariances @ weights
        return r, sd #type: ignore

    def solve_analytical(self):
        n, = self.expecteds.shape
        A = np.array(self.covariances)
        b = np.ones((n,))

        A_inv: NDArray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n] / np.sum(x[:n])
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

    def solve_analytical_target_return(self, target_mean_return: float):
        """Solves the Markowitz problem analytically. Enter the target mean return r-bar in percentage"""
        r_bar = target_mean_return
        n, = self.expecteds.shape

        # There are n + 2 variables in total, the equations (in compact form) are:
        # w1 + ... + wn = 1
        # w1r1 + w2r2 + ... + wnrn = r_bar
        # C w - l r - m = 0
        # where the n + 2 variables are w1 ... wn and l and m

        # So in the spirit of solving system of linear equations, we make the matrix A and vector b so that we solve Ax = b

        A = np.zeros((n+2, n+2))
        b = np.zeros((n+2,))

        # First equation
        A[0, :n] = 1
        b[0] = 1

        # Second equation
        A[1, :n] = self.expecteds
        b[1] = r_bar

        # Covariance equations
        A[2:, :n] = self.covariances
        A[2:, n] = -self.expecteds
        A[2:, n+1] = -1

        # Solve equation the lazy way:
        A_inv: NDArray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n]
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

    def solve_tan_analytical(self, risk_free_rate: float):
        n, = self.expecteds.shape
        A = np.array(self.covariances)
        b = self.expecteds - risk_free_rate

        A_inv: NDArray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n] / np.sum(x[:n])
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

def markowitz_analytical(expected_returns, covariances):
    """Solves the classical markowitz problem by using approximation approach"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_analytical()

def markowitz_riskfree_analytical(expected_returns, covariances, risk_free):
    """Solves markowitz problem with risk free rate. Enter risk free rate in percentage"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_tan_analytical(risk_free)

def markowitz_fixed_mean_analytical(expected_returns, covariances, target_mean_return):
    """Solves markowitz problem with a target mean return. Enter risk free rate in percentage"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_analytical_target_return(target_mean_return)

In [ ]:
## Q9: Kill me
returns = np.array([69, 65, 44, 36, 41])

covs = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

print("Q9 - aim for 80% return")
w = markowitz_fixed_mean_analytical(returns, covs, 80)
print(w[0] * 10000)

print("\n\nQ10")
markowitz_analytical(returns, covs)

print("\n\nQ11")
w = markowitz_riskfree_analytical(returns, covs, 4)
print(w[3])


Q9 - aim for 80% return
Best average return: 79.99999999999996
Minimum standard deviation: 0.1618979763761178
Best weights: [ 0.74492302  0.89403309 -0.5980176   0.30411715 -0.34505565]
7449.230162571025


Q10
Best average return: 53.00171526586621
Minimum standard deviation: 0.12381360777587187
Best weights: [ 0.0971984   0.53230417 -0.15208691  0.60777587 -0.08519154]


Q11
Best average return: 101.36131335760292
Minimum standard deviation: 0.2460047653270432
Best weights: [ 1.25740922  1.18023663 -0.95084243  0.06385933 -0.55066275]
0.0638593297862414


Q12

In [ ]:
betas = np.array([1.16, 0.70, 1.15, 0.98, 1.01])

# Let's say we want to do something about factor models
# Each stock return is modelled by ri = ai + bi

Question 11


In [ ]:
# Covariance matrix
cov_matrix = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

# Mean returns
mean_returns = np.array([0.69, 0.65, 0.44, 0.36, 0.41])

def rf_mini_var_2asset(V, r_bar, rf):
    v1 = np.linalg.solve(V, np.ones(len(V)))
    v2 = np.linalg.solve(V, r_bar)
    w1 = v1/sum(v1)
    w2 = v2/sum(v2)
    v  = v2 - rf*v1
    w  = v/sum(v)
    return print(f"v1 = {v}, w1 = {w1}\nv2 = {v2}, w2 = {w2}\nv  = {v}, w = {w}")


rf_mini_var_2asset(cov_matrix, mean_returns, rf=0.04)   # rf = 4% and can be set as 0 

v1 = [ 4.97644886  4.67102286 -3.76314939  0.25273609 -2.17935812], w1 = [ 0.0971984   0.53230417 -0.15208691  0.60777587 -0.08519154]
v2 = [ 5.00785038  4.84299238 -3.81228354  0.44908797 -2.20688063], w2 = [ 1.16984898  1.13133765 -0.89056095  0.10490831 -0.51553398]
v  = [ 4.97644886  4.67102286 -3.76314939  0.25273609 -2.17935812], w = [ 1.25740922  1.18023663 -0.95084243  0.06385933 -0.55066275]


Practice Final Question 14
Find YTM first, Plug back to finde price
https://www.omnicalculator.com/finance/yield-to-maturity

https://www.omnicalculator.com/finance/bond-price

Practice Final Question 15

In [ ]:
# Invest to pay off mortgage FPQ15
# Terms!!! Make sure it in month or year!!!
def inv_payoff(int_rate,terms,growth_rate,priciple):
  M = priciple*(int_rate/(1-(1+int_rate)**(-terms)))
  PFV = M*(((1+growth_rate)**terms)-1)/growth_rate
  FV_I = priciple*(1+growth_rate)**terms
  FV = FV_I - PFV
  print(FV)
  return FV


In [ ]:
inv_payoff(0.0025,360,0.01,500000)

10607365.783267662


10607365.783267662

Question 16
100 mutually uncorrelated assets


In [ ]:
def sum_sharpe_ratios(num_assets):
    return sum(range(1, num_assets + 1))

def optimal_portfolio_weight(num_assets):
    return 1 / sum_sharpe_ratios(num_assets)

num_assets = 100

optimal_weight = optimal_portfolio_weight(num_assets)
optimal_weight_percentage = optimal_weight * 100
optimal_max_sharpe = optimal_weight_percentage * 100

print("The optimal portfolio weight for the risky asset with the maximum Sharpe ratio is approximately: optimal_max_sharpe:",optimal_max_sharpe)


The optimal portfolio weight for the risky asset with the maximum Sharpe ratio is approximately: optimal_max_sharpe: 1.9801980198019802


Practice Final Question 17

In [ ]:
# constant-proportion rebalancing FPQ17

def Cons_propo_rebal(prop_of_cash,prop_of_stock,days,initial):
  stock = prop_of_stock * initial
  cash = prop_of_cash * initial
  for i in range(10):
    Ex = ((stock)/2 + 2*(stock))/2
    stock = prop_of_stock*(Ex+cash)
    cash = prop_of_cash*(Ex+cash)
  E = stock + cash
  print(E)
  return E

Cons_propo_rebal(0.5,0.5,10,100)

324.7321025468409


324.7321025468409

Q18: fuuck me

In [ ]:
from scipy.optimize import minimize

bonds = [
    Bond(0, 5, 1, 100),
    Bond(0, 10, 1, 100),
    Bond(0, 15, 1, 100),
    Bond(0, 20, 1, 100),
    Bond(0, 25, 1, 100),
    Bond(0, 30, 1, 100)
]

yield_curves = [1, 2.5, 2.5, 3, 3, 3.1]

def bond_price(bond, yield_to_maturity: float, remaining_periods = None):
    """Calculate the price of the bond based on its yield to maturity. The price and yield operations should be inverses to each other, given the same number of remaining periods until maturity.
    Please enter your yield to maturity in percentages"""
    if remaining_periods is None:
        remaining_periods = bond.coupon_freq * bond.maturity_years
    
    m = bond.coupon_freq
    F = bond.face_value
    C = bond.percentage * bond.face_value
    y = yield_to_maturity / 100
    n = remaining_periods

    fac = 1 / (1 + y / m) ** n
    return F * fac + C / y * (1 - fac)

def treasury_yield_my_ass(bonds, yield_curves):
    prices = [bond_price(bond, rate) for bond, rate in zip(bonds, yield_curves)]

    # Lets say we have these weights
    s = [1, 0, 0, 0, 0, 0]

    def optimize_this(s):
        cost = sum(a * b for a, b in zip(s, prices))

        # In five years let's say we sell eveything
        y5 = 100 * s[0]
        for i in range(5):
            y5 += s[i+1] * prices[i]
        # We want biggest profit/cost ratio, hence smallest cost/profit
        return cost/y5

    ms = minimize(optimize_this, [1, 0, 0, 0, 0, 0], bounds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)])

    weights = ms.x
    return 1/optimize_this(weights)

five_year_max_return_ratio = treasury_yield_my_ass(bonds, yield_curves)

# Do the five year thing twice
1e5 * five_year_max_return_ratio * five_year_max_return_ratio

155515.26561194478

Hard one

In [2]:
import numpy as np
from scipy.optimize import minimize

# Mean returns
mean_returns = np.array([0.69, 0.65, 0.44, 0.36, 0.41])

# Covariance matrix
cov_matrix = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

def objective_function(w):
    return np.dot(np.dot(w, cov_matrix), w) - np.dot(w, mean_returns)

def constraint_function(w):
    return np.sum(w) - 1

# Initial guess for weights
initial_weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

# Bounds for weights (allowing for short positions)
bounds = [(-10.0, 10.0)] * 5

# Constraint
constraints = {'type': 'eq', 'fun': constraint_function}

# Optimization
result = minimize(objective_function, initial_weights, bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)

Optimal weights: [ 2.39275041  1.81502333 -1.73234323 -0.46878321 -1.0066473 ]
